## Commands

In [1]:
# oomstopper --no-tail genericmodels ; killbill genericmodels ; cd ~/twinews-logs ; jupython -o nohup-genericmodels-$HOSTNAME.out --venv st-venv ~/Workspace/Python/Datasets/Twinews/twinews/models/genericmodels.ipynb

## Init

In [2]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
isNotebook = '__file__' not in locals()

In [4]:
TEST = isNotebook # isNotebook, True, False

In [5]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from nlptools.preprocessing import *
from nlptools.basics import *
from twinews.utils import *
from twinews.models.ranking import *
from machinelearning.iterator import *

In [6]:
from twinews.models.genericutils import *

In [7]:
logger = Logger(tmpDir('logs') + "/generic.log") if isNotebook else Logger("generic-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

## Config

In [8]:
cacheKey = \
(
    # 'dbert-ft'
    # 'dbert-base'
    # 'infersent'
    # 'usent'
    # 'sent2vec'
    'doc2vec'
    # 'bert'
    # 'stylo'
    # 'nmf'
    # 'tfidf'
    # 'word2vec'
)
cacheField = genericFields[cacheKey]

In [9]:
config = \
{
    'splitVersion': 2,
    'maxUsers': 2 if TEST else None, # Sub-sampling
    
    'distance': 'cosine', # 'cosine', 'euclidean', 'kl', 'js' (ValueError math domain error pour kl et js)
    'historyRef': 0.35, # 0.1, 0.3, 0.4, 0.6, 1.0, 1, 3, 10, 30
}

In [10]:
# config = mergeDicts(config, {})
modelName = cacheKey
getCache = lambda: getGenericCache(cacheKey, verbose=False)

In [11]:
# Here we check if we already generated ranking for this model with this specific config:
if not isNotebook:
    if rankingExists(modelName, config, logger=logger):
        raise Exception(modelName + " with this config already exist:\n" + b(config, 5))

In [12]:
log("Generating rankings for " + modelName + ' (`' + cacheField + '` field) with the config:\n' + b(config, 5), logger)

Generating rankings for infersent (`detokSentences` field) with the config:
{ 'distance': cosine, 'historyRef': 0.4, 'maxUsers': 2, 'splitVersion': 2 }


## Getting data

In [13]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0, maxUsers=config['maxUsers'], logger=logger)
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)

--> tic: 32.66s | message: Eval data loaded
--> toc total duration: 32.97s | message: Got Twinews evaluation data
{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }
{ 'created': 2020.03.24-14.28.06, 'endDate': 2018-01-15, 'id': 2, 'ranksLength': 1000, 'splitDate': 2017-12-25, 'startDate': 2017-10-01, 'testMaxNewsPerUser': 97, 'testMeanNewsPerUser': 7.22, 'testMinNewsPerUser': 2, 'testNewsCount': 71781, 'totalNewsAvailable': 570210, 'trainMaxNewsPerUser': 379, 'trainMeanNewsPerUser': 26.48, 'trainMinNewsPerUser': 8, 'trainNewsCount': 237150, 'usersCount': 15905 }


In [14]:
# Here it is important to convert urls to lists because we want the same order to retrieve vectors by index...
# And we shuffle it so we do not stick urls a a user at the begin...
# But we seed the random to always have same order...
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)
newsList = trainNewsList + testNewsList

In [15]:
# Print all:
log(str(len(trainNewsList)) + " urls for trainNewsList", logger=logger)
log(str(len(testNewsList)) + " urls for testNewsList", logger=logger)
log(str(len(newsList)) + " urls for newsList", logger=logger)

28 urls for trainNewsList
1989 urls for testNewsList
2017 urls for newsList


In [16]:
def genFunct(containers, field, getCache, *args, **kwargs):
    if not isinstance(containers[0], list):
        containers = [containers]
    cache = getCache()
    newsCollection = getNewsCollection(verbose=False)
    for container in containers:
        for url in container:
            vector = getVector(url, field, cache, newsCollection)
            assert vector is not None
            yield (url, vector)

In [17]:
mli = MLIterator\
(
    chunks(newsList, 100 if TEST else 5000),
    # chunks(newsList, int(len(newsList) / 100)),
    genFunct, genArgs=(cacheField, getCache,),
    logger=logger,
    printRatio=0.03, queuesMaxSize=1000,
    parallelProcesses=cpuCount(),
)

With parallelProcesses > 1, this iterator is not consistent, meaning 2 iterations over same containers will not give items in the same order
21 containers to process.


In [18]:
urlsVectors = dict()
for url, vector in mli:
    urlsVectors[url] = vector

Exception type: <class 'AssertionError'>
Exception: 
Traceback (most recent call last):
  File "/home/hayj/Workspace/Python/Utils/MachineLearning/machinelearning/iterator.py", line 90, in genFunctWrapper
    for current in genFunct(container, *genArgs, **genKwargs, logger=logger, verbose=verbose):
  File "<ipython-input-16-2fbc26184d0b>", line 9, in genFunct
    assert vector is not None
AssertionError

Exception type: <class 'AssertionError'>
Exception: 
Traceback (most recent call last):
  File "/home/hayj/Workspace/Python/Utils/MachineLearning/machinelearning/iterator.py", line 90, in genFunctWrapper
    for current in genFunct(container, *genArgs, **genKwargs, logger=logger, verbose=verbose):
  File "<ipython-input-16-2fbc26184d0b>", line 9, in genFunct
    assert vector is not None
AssertionError

Exception type: <class 'AssertionError'>
Exception: 
Traceback (most recent call last):
  File "/home/hayj/Workspace/Python/Utils/MachineLearning/machinelearning/iterator.py", line 90, i

KeyboardInterrupt: 

In [ ]:
bp(urlsVectors, logger)

In [ ]:
tt.tic("Data loaded")

## Ranking

In [ ]:
# Read the doc!
rankings = usersRankingsByHistoryDistance\
(
    trainUsers,
    candidates,
    config['historyRef'],
    urlsVectors,
    distanceMetric=config['distance'],
    logger=logger,
)

In [ ]:
bp(rankings, logger, 4)

In [ ]:
tt.tic("Rankings done")

## Adding rankings to the db

In [ ]:
# Read the doc!
addRanking(modelName, rankings, config, logger=logger)

In [ ]:
tt.tic("Rankings stored")

In [ ]:
totalDuration = tt.toc()

In [ ]:
if not TEST:
    notif(modelName + '-' + objectToHash(config)[:5] + " done in " + secondsToHumanReadableDuration(totalDuration) + " on " + getHostname())

## Old stuff

In [ ]:
if False:
    (user, password, host) = getMongoAuth(user='hayj')
    cache = SerializableDict\
    (
        "twinews-dbert-94bef_ep32",
        user=user, host=host, password=password,
        useMongodb=True, logger=logger,
    )
    urlCache = SerializableDict\
    (
        "twinews-dbert-94bef_ep32-url",
        user=user, host=host, password=password,
        useMongodb=True, logger=logger,
    )
    pbar = ProgressBar(len(newsCollection), logger=logger, printRatio=0.01)
    for row in newsCollection.find({}, projection={'detokText': True, 'url': True}):
        url = row['url']
        text = row['detokText']
        theHash = objectToHash(text)
        vector = cache[theHash]
        urlCache[url] = vector
        pbar.tic()
    exit()